In [1]:
import os
import pickle

import skimage
from skimage import data
import numpy as np
from numpy import matlib
from sklearn.datasets import load_svmlight_file
from PIL import Image
import matplotlib.pylab as plt
from matplotlib.pyplot import imshow

import scfw.poisson as ps
from scfw.scopt import scopt
from scfw.prox_grad import prox_grad

In [2]:
%matplotlib inline

## Data load

In [3]:
data_folder = '../data'

problem_name = 'a1a'
results = {problem_name: {}}
W, y = load_svmlight_file(os.path.join(data_folder, problem_name))
y = y.reshape(-1, 1)
W = W.toarray()

results_folder = './results/'
results_file=os.path.join(results_folder, 'poisson', problem_name+'.pckl')
results = {problem_name: {}}
if os.path.isfile(results_file):
    with open(results_file, "rb") as f:
        results=pickle.load(f)

## Data preprocessing

In [4]:
# multiplicative multiplication of the structure of the recognizable structure
W = matlib.repmat(y, 1, W.shape[1]) * W
# summation of feature descriptions
sA = np.sum(W, 1)
# if the sum of the string is negative, invert it
W[sA < 0, :] = -W[sA < 0, :]
# zero row deletion
W = W[sA != 0, :]
# W = scipy.sparse.csr_matrix(W)

Bias = 1
b = (Bias * y).squeeze()
b = np.abs(b)
if (b.any == 0):
    print(' Input parameter y error')

y = b
N, n = W.shape

## Parameters initialization

In [5]:
lam =  np.sqrt(N)/2
#first set of parameters
Mf = max(2 / np.sqrt(y))
nu = 3
I = np.eye(n)
Iv = np.ones(n)
#running parameters
x0 = np.ones(n) / n
terminate_tol = 1e-10
M = ps.poisson_matr(W, y, lam, x0)[0] / lam

FW_params={
    'iter_FW': 50000,
    'line_search_tol': 1e-10,
}

prox_params={
        #parameters for SCOPT
        'iter_prox': 1000,
        'Lest': 'estimate',#,'estimate', #estimate L
        'bb_type': 3,
        #FISTA parameters
        'fista_type': 'fista',
        'fista_tol': 1e-5,
        'fista_iter': 1000,
        'btk_iters': 100,
        'backtracking': True
    }

sc_params={
    #parameters for SCOPT
    'iter_SC': 1000,
    'Lest': 'estimate', #estimate L
    'use_two_phase': True,
    #FISTA parameters
    'fista_type': 'mfista',
    'fista_tol': 1e-5,
    'fista_iter': 1000,
    #Conjugate Gradient Parameters
    'conj_grad_tol': 1e-2,
    'conj_grad_iter': 100,
}

## Auxilary functions

In [6]:
func_x = lambda x: ps.poisson_matr(W, y, lam, x)
grad_x = lambda x, dot_product: ps.grad_poisson_matr(W, y, lam, x, dot_product)
grad_beta = lambda x, s, beta, dot_product, dot_product_s: ps.grad_poisson_matr(W, y, lam, (1 - beta) * x + beta * s, (1 - beta) * dot_product + beta * dot_product_s)
hess_mult_x = lambda x, dot_product: ps.hess_mult_matr(W, y, x, dot_product)
hess_mult_vec_x = lambda s, dot_product: ps.hess_mult_vec_matr(W, y, s, dot_product)
extra_func = lambda x: W @ x
linear_orcale = lambda grad: ps.linear_oracle_full_simplex(grad, M)
prox_func = lambda x, L: np.maximum(x, 0) #used for SCOPT

## Run SCOPT

In [7]:
%%time
x, alpha_hist, Q_hist, time_hist = scopt(func_x,
          grad_x,
          hess_mult_x,
          hess_mult_vec_x,
          Mf,
          nu,
          prox_func,
          x0,  
          sc_params,                                              
          eps=terminate_tol,                                              
          print_every=10)
    
results[problem_name]['scopt'] = {
    'x': x,
    'alpha_hist': alpha_hist,
    #'Gap_hist': Gap_hist,
    'Q_hist': Q_hist,
    'time_hist': time_hist,
}

iter =    1, stepsize = 2.728e-02, rdiff = 1.027e-01 , f = 3658.74
iter =   10, stepsize = 2.832e-02, rdiff = 9.296e-02 , f = 3356.65
iter =   20, stepsize = 2.980e-02, rdiff = 8.464e-02 , f = 3035.05
iter =   30, stepsize = 3.176e-02, rdiff = 8.572e-02 , f = 2731.84
iter =   40, stepsize = 3.464e-02, rdiff = 9.074e-02 , f = 2451.68
iter =   50, stepsize = 3.807e-02, rdiff = 1.904e-01 , f = 2199.88
iter =   60, stepsize = 4.610e-02, rdiff = 2.271e-01 , f = 1981.61
iter =   70, stepsize = 5.859e-02, rdiff = 1.335e-01 , f = 1809.1
iter =   80, stepsize = 9.745e-02, rdiff = 1.799e-01 , f = 1693.23
iter =   90, stepsize = 3.178e-01, rdiff = 7.247e-02 , f = 1643
boo
Convergence achieved!
iter =   98, stepsize = 1.000e+00, rdiff = 9.442e-15,value=1640.16
13.643610000610352
Wall time: 13.6 s


In [8]:
%%time
x, alpha_hist, Q_hist, time_hist = prox_grad(func_x,
                grad_x,
                prox_func,
                Mf,
                x0,
                prox_params,
                eps=terminate_tol,
                print_every=100)

 rdiff = 3.148e+00 , f = 1644.96
iter =  703, stepsize = 1.333e-02, rdiff = 4.384e-01 , f = 1646.4
iter =  704, stepsize = 1.288e-01, rdiff = 5.806e-01 , f = 1645.46
iter =  705, stepsize = 4.810e-03, rdiff = 9.254e-01 , f = 1650.62
iter =  706, stepsize = 4.814e-03, rdiff = 6.178e+00 , f = 1649.05
iter =  707, stepsize = 4.681e-02, rdiff = 1.205e-01 , f = 1644.23
iter =  708, stepsize = 6.840e-02, rdiff = 2.588e+00 , f = 1643.99
iter =  709, stepsize = 9.198e-03, rdiff = 6.691e-01 , f = 1696.68
iter =  710, stepsize = 9.509e-02, rdiff = 6.638e-01 , f = 1692.58
iter =  711, stepsize = 2.226e-02, rdiff = 6.007e-01 , f = 1659.01
iter =  712, stepsize = 4.456e-02, rdiff = 5.873e-01 , f = 1654.06
iter =  713, stepsize = 2.413e-02, rdiff = 5.524e-01 , f = 1646.81
iter =  714, stepsize = 4.212e-02, rdiff = 5.410e-01 , f = 1644.41
iter =  715, stepsize = 9.542e-02, rdiff = 2.601e-01 , f = 1642.59
iter =  716, stepsize = 1.212e-02, rdiff = 1.131e+00 , f = 1643.41
iter =  717, stepsize = 7.714e

## Run prox grad

In [8]:
def run_prox_grad(problem_name):
    results = {problem_name: {}}
    W, y = load_svmlight_file(os.path.join(data_folder, problem_name))
    y = y.reshape(-1, 1)
    W = W.toarray()
    results_file=os.path.join(results_folder, 'poisson', problem_name+'.pckl')
    results = {problem_name: {}}
    if os.path.isfile(results_file):
        with open(results_file, "rb") as f:
            results=pickle.load(f)
    # multiplicative multiplication of the structure of the recognizable structure
    W = matlib.repmat(y, 1, W.shape[1]) * W
    # summation of feature descriptions
    sA = np.sum(W, 1)
    # if the sum of the string is negative, invert it
    W[sA < 0, :] = -W[sA < 0, :]
    # zero row deletion
    W = W[sA != 0, :]
    # W = scipy.sparse.csr_matrix(W)
    Bias = 1
    b = (Bias * y).squeeze()
    b = np.abs(b)
    if (b.any == 0):
        print(' Input parameter y error')
    y = b
    N, n = W.shape
    lam =  np.sqrt(N)/2
    Mf = max(2 / np.sqrt(y))
    nu = 3
    I = np.eye(n)
    Iv = np.ones(n)
    x0 = np.ones(n) / n
    terminate_tol = 1e-10
    M = ps.poisson_matr(W, y, lam, x0)[0] / lam
    func_x = lambda x: ps.poisson_matr(W, y, lam, x)
    grad_x = lambda x, dot_product: ps.grad_poisson_matr(W, y, lam, x, dot_product)
    grad_beta = lambda x, s, beta, dot_product, dot_product_s: ps.grad_poisson_matr(W, y, lam, (1 - beta) * x + beta * s, (1 - beta) * dot_product + beta * dot_product_s)
    hess_mult_x = lambda x, dot_product: ps.hess_mult_matr(W, y, x, dot_product)
    hess_mult_vec_x = lambda s, dot_product: ps.hess_mult_vec_matr(W, y, s, dot_product)
    extra_func = lambda x: W @ x
    linear_orcale = lambda grad: ps.linear_oracle_full_simplex(grad, M)
    prox_func = lambda x, L: np.maximum(x, 0) #used for SCOPT
    prox_params={
        #parameters for SCOPT
        'iter_prox': 50000,
        'Lest': 'estimate',#,'estimate', #estimate L
        'bb_type': 2,
        #FISTA parameters
        'fista_type': 'fista',
        'fista_tol': 1e-5,
        'fista_iter': 1000,
        'btk_iters': 100,
        'backtracking': True
    }

    x, alpha_hist, Q_hist, time_hist = prox_grad(func_x,
                grad_x,
                prox_func,
                Mf,
                x0,
                prox_params,
                eps=terminate_tol,
                print_every=10000)

    results[problem_name]['prox_grad'] = {
        'x': x,
        'alpha_hist': alpha_hist,
        'Q_hist': Q_hist,
        'time_hist': time_hist,
    }
    with open(results_file, 'wb') as f:
        pickle.dump(results, f)

In [9]:
data_list =  ['a1a','a2a','a3a', 'a4a', 'a5a', 'a6a','a7a','a8a','a9a']
for problem_name in data_list:
    run_prox_grad(problem_name)

iter =    1, stepsize = 6.602e-05, rdiff = 1.833e-01 , f = 3658.74
iter = 10000, stepsize = 5.477e-04, rdiff = 1.951e-01 , f = 1933.45
Convergence achieved!
iter = 11843, stepsize = 1.000e+00, rdiff = 8.894e-11,value=1640.16
0.0
iter =    1, stepsize = 4.678e-05, rdiff = 9.167e-02 , f = 5158.41
iter = 10000, stepsize = 1.250e-04, rdiff = 1.529e-01 , f = 3682.06
Convergence achieved!
iter = 16530, stepsize = 1.000e+00, rdiff = 2.198e-11,value=2307.58
0.0
iter =    1, stepsize = 3.240e-05, rdiff = 9.054e-02 , f = 7318.45
iter = 10000, stepsize = 5.627e-05, rdiff = 5.928e-02 , f = 6054
iter = 20000, stepsize = 2.235e-04, rdiff = 9.841e-02 , f = 3982.58
Convergence achieved!
iter = 23966, stepsize = 1.000e+00, rdiff = 7.837e-11,value=3236.42
0.0009980201721191406
iter =    1, stepsize = 2.156e-05, rdiff = 1.811e-01 , f = 10978.1
iter = 10000, stepsize = 3.016e-05, rdiff = 5.125e-02 , f = 9791.35
iter = 20000, stepsize = 4.962e-05, rdiff = 7.007e-02 , f = 8236.28
iter = 30000, stepsize = 1.

In [7]:
prox_params={
        #parameters for SCOPT
        'iter_prox': 50000,
        'Lest': 'estimate',#,'estimate', #estimate L
        'bb_type': 1,
        #FISTA parameters
        'fista_type': 'fista',
        'fista_tol': 1e-5,
        'fista_iter': 1000,
    }

x, alpha_hist, Q_hist, time_hist = prox_grad(func_x,
            grad_x,
            hess_mult_vec_x,
            prox_func,
            Mf,
            x0,
            prox_params,
            eps=terminate_tol,
            print_every=1000)

results[problem_name]['prox_grad'] = {
    'x': x,
    'alpha_hist': alpha_hist,
    'Q_hist': Q_hist,
    'time_hist': time_hist,
}

iter =    1, stepsize = 6.343e-05, rdiff = 1.592e+01 , f = 3658.74
iter = 1000, stepsize = 7.845e-05, rdiff = 9.239e-02 , f = 3452.16
iter = 2000, stepsize = 9.291e-05, rdiff = 1.043e-01 , f = 3256.85
iter = 3000, stepsize = 1.139e-04, rdiff = 1.192e-01 , f = 3035.15
iter = 4000, stepsize = 1.470e-04, rdiff = 1.384e-01 , f = 2779.97
iter = 5000, stepsize = 2.070e-04, rdiff = 1.626e-01 , f = 2482.21
iter = 6000, stepsize = 3.493e-04, rdiff = 1.883e-01 , f = 2134.04
iter = 7000, stepsize = 1.096e-03, rdiff = 1.634e-01 , f = 1765.1
iter = 8000, stepsize = 1.000e+00, rdiff = 4.605e-07 , f = 1640.16
iter = 9000, stepsize = 1.000e+00, rdiff = 1.815e-08 , f = 1640.16
iter = 10000, stepsize = 1.000e+00, rdiff = 4.908e-09 , f = 1640.16
iter = 11000, stepsize = 1.000e+00, rdiff = 2.519e-09 , f = 1640.16
iter = 12000, stepsize = 1.000e+00, rdiff = 1.205e-09 , f = 1640.16
iter = 13000, stepsize = 1.000e+00, rdiff = 1.168e-09 , f = 1640.16
iter = 14000, stepsize = 1.000e+00, rdiff = 7.506e-10 , f =

In [8]:
with open(results_file, 'wb') as f:
    pickle.dump(results, f)

In [8]:
results['a1a'].keys()

dict_keys(['standard', 'line_search', 'backtracking', 'sc', 'sc_backtracking', 'sc_hybrid', 'scopt', 'prox_grad'])